In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo DEBIAN_FRONTEND=noninteractive apt-get install -y cuda-9.0

--2024-11-27 17:41:43--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.nvidia.com/downloads/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb [following]
--2024-11-27 17:41:44--  https://developer.nvidia.com/downloads/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?Z9u8mxXj6O3U9-rwzyyrA1WUZ2_r1PohMpnHv8jNqKc9ZaDxtWuWFZIyR8uELpLN70qA09z-MfSz8b4slKQOX

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


In [ ]:
!pip install thundersvm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.4/507.4 kB 9.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from thundersvm import SVC

In [ ]:
from zipfile import ZipFile
with ZipFile('Data/2021_stat.zip', 'r') as zipObj:
  zipObj.extractall('Data')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/MIE479/Deliverable_4')
!ls '/content/drive/MyDrive/MIE479/Deliverable_4'

'SVM Models.ipynb'


In [ ]:
file_name = '/content/Data/2021_stat.csv'
table = pd.read_csv(file_name, low_memory=False).dropna()

table = table[[
    'ORIG_RATE', 'ORIG_AMOUNT', 'ORIG_TERM', 'OLTV', 'NUM_BO', 'DTI', 'CSCORE_B', # 'CSCORE_C',
    'FTHB_FLAG', 'PUR_Cash_out', 'PUR_Refinance', 'PUR_Purchase', 'NUM_UNIT',
    'OCC_Principal', 'OCC_Second', 'OCC_Investor', 'MI_PCT', 'DLQ_30_FLAG', 'DLQ_90_FLAG',
    'Ongoing', 'Current_DLQ', 'Prepaid_Matured', 'Deed_In_Lieu']]

table['BAD'] = None
table.loc[(table['DLQ_90_FLAG'] == 1) | (table['Deed_In_Lieu'] == 1), 'BAD'] = 1
table.loc[(table['Ongoing'] == 1) & (table['DLQ_30_FLAG'] == 0), 'BAD'] = 0
table = table.dropna(subset=['BAD'])
table['BAD'] = table['BAD'].astype(int)

<ipython-input-7-50234218f0af>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['BAD'] = table['BAD'].astype(int)


In [ ]:
n_row, n_col = table.shape
n_dlq_90 = sum(table['DLQ_90_FLAG'])
# n_prepay = sum(table['Prepaid_Matured'])
n_fc = sum(table['Deed_In_Lieu'])
print(f"Number of rows: {n_row}")
print(f"Number of columns: {n_col}")
print(f"Percentage of 90-day delinquency: {round(n_dlq_90 / n_row * 100, 2)} %")
# print(f"Percentage of prepayment: {round(n_prepay / n_row * 100, 2)} %")
print(f"Percentage of foreclosure: {round(n_fc / n_row * 100, 2)} %")
print("The total number of NA is ", sum(table.isna().sum()))

Number of rows: 3896571
Number of columns: 28
Percentage of 90-day delinquency: 1.45 %
Percentage of foreclosure: 0.01 %
The total number of NA is  0


In [ ]:
sampled = table.sample(n=2000000, random_state=999)

X = sampled.drop(columns=['DLQ_30_FLAG', 'DLQ_90_FLAG', 'Ongoing', 'Current_DLQ',
                        'Prepaid_Matured', 'Deed_In_Lieu', 'BAD'])
y = sampled['BAD']

num_col = ['ORIG_RATE', 'ORIG_AMOUNT', 'ORIG_TERM', 'OLTV', 'NUM_BO', 'DTI', 'CSCORE_B', # 'CSCORE_C',
           'NUM_UNIT', 'MI_PCT']
cat_col = ['FTHB_FLAG', 'PUR_Cash_out', 'PUR_Refinance', 'PUR_Purchase',
           'OCC_Principal', 'OCC_Second', 'OCC_Investor']

scaler = StandardScaler()
X[num_col] = scaler.fit_transform(X[num_col])
X = pd.concat([X[num_col], X[cat_col]], axis=1)

X

,ORIG_RATE,ORIG_AMOUNT,ORIG_TERM,OLTV,NUM_BO,DTI,CSCORE_B,NUM_UNIT,MI_PCT,FTHB_FLAG,PUR_Cash_out,PUR_Refinance,PUR_Purchase,OCC_Principal,OCC_Second,OCC_Investor
4013967,0.228753,1.777391,0.617818,1.467358,-0.923561,0.150089,-0.185992,-0.109492,2.375319,0,0,0,1,1,0,0
308387,0.822908,-0.413166,0.617818,0.507244,-0.923561,1.255463,-0.256433,-0.109492,-0.467726,0,0,1,0,1,0,0
3254368,0.822908,0.168701,-0.145503,0.987301,-0.923561,-0.955284,0.048811,-0.109492,1.901478,0,0,1,0,1,0,0
2197392,0.204987,-0.461085,-0.908823,-0.186172,-0.923561,-1.055773,0.189692,-0.109492,-0.467726,0,1,0,0,1,0,0
429610,-2.171629,-0.714368,-1.672144,-1.412984,1.010637,0.351067,1.034982,-0.109492,-0.467726,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168686,-0.674361,0.120782,-0.908823,0.080527,-0.923561,0.953998,0.166212,-0.109492,-0.467726,0,1,0,0,1,0,0
3028270,0.228753,0.490439,0.617818,1.574037,-0.923561,-0.050888,0.847140,-0.109492,2.849160,1,0,0,1,1,0,0
819887,-0.959555,-0.426857,0.617818,0.667263,1.010637,0.652532,0.823660,-0.109492,-0.467726,0,0,0,1,1,0,0
3884724,1.131868,-0.057201,-1.672144,0.667263,1.010637,0.552044,-2.205295,-0.109492,-0.467726,0,1,0,0,1,0,0


In [ ]:
### Don't run linear kernel with Thunder SVM

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)

linear_svm = SVC(kernel='linear', class_weight='balanced', max_iter=50000)
linear_svm.fit(X_train, y_train)

y_pred = linear_svm.predict(X_test)
y_pred_prob = linear_svm.decision_function(X_test)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
roc_auc_prob = roc_auc_score(y_test, y_pred_prob)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"AUC-ROC Predict: {round(roc_auc, 4)}")
print(f"AUC-ROC Decision function: {round(roc_auc_prob, 4)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7296
AUC-ROC Predict: 0.7492
AUC-ROC Decision function: 0.1788
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.73      0.84    591453
           1       0.04      0.77      0.07      8547

    accuracy                           0.73    600000
   macro avg       0.52      0.75      0.46    600000
weighted avg       0.98      0.73      0.83    600000



In [ ]:
print(round(y_test.sum() / len(y_test), 4))
print(round(y_pred.sum() / len(y_pred), 4))

0.0142
0.278


In [ ]:
coefficients = linear_svm.coef_
intercept = linear_svm.intercept_
print("Coefficients (w):")
print(coefficients)
print("Intercept (b):")
print(intercept)
sv = linear_svm.support_vectors_
print(len(sv))

Coefficients (w):
[[-0.12206957 -0.07152963 -0.08705504 -0.18176843  0.27858248 -0.26226771
   0.71115422 -0.02490039 -0.15293053 -0.03796965 -0.06215479  0.01679347
   0.04536132 -0.13183662  0.02536996  0.10646667]]
Intercept (b):
[-0.76651949]
807096


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)
gamma_value = 1 / (X_train.to_numpy().shape[1] * np.var(X_train.to_numpy()))

rbf_svm = SVC(kernel='rbf', C=1.0, gamma=gamma_value, class_weight='balanced')
rbf_svm.fit(X_train, y_train)

y_pred = rbf_svm.predict(X_test)
y_pred_prob = rbf_svm.decision_function(X_test)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
roc_auc_prob = roc_auc_score(y_test, y_pred_prob)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"AUC-ROC Predict: {round(roc_auc, 4)}")
print(f"AUC-ROC Decision function: {round(roc_auc_prob, 4)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7335
AUC-ROC Predict: 0.7512
AUC-ROC Decision function: 0.1967
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.73      0.84    591453
           1       0.04      0.77      0.08      8547

    accuracy                           0.73    600000
   macro avg       0.52      0.75      0.46    600000
weighted avg       0.98      0.73      0.83    600000



In [ ]:
print(round(y_test.sum() / len(y_test), 4))
print(round(y_pred.sum() / len(y_pred), 4))

import pickle
with open('/content/drive/MyDrive/MIE479/Deliverable_4/rbf_svm_2021.pkl', 'wb') as file:
    pickle.dump(rbf_svm, file)

rbf_vectors = rbf_svm.support_vectors_
print(len(rbf_vectors))

0.0142
0.2978
719917


In [ ]:
poly_svm = SVC(kernel='polynomial', C=1, gamma=gamma_value, degree=2, coef0=1, class_weight='balanced')
poly_svm.fit(X_train, y_train)

y_pred = poly_svm.predict(X_test)
y_pred_prob = poly_svm.decision_function(X_test)


accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
roc_auc_prob = roc_auc_score(y_test, y_pred_prob)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"AUC-ROC Predict: {round(roc_auc, 4)}")
print(f"AUC-ROC Decision function: {round(roc_auc_prob, 4)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7105
AUC-ROC Predict: 0.7506
AUC-ROC Decision function: 0.1783
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.71      0.83    591453
           1       0.04      0.79      0.07      8547

    accuracy                           0.71    600000
   macro avg       0.52      0.75      0.45    600000
weighted avg       0.98      0.71      0.82    600000



In [ ]:
print(round(y_test.sum() / len(y_test), 4))
print(round(y_pred.sum() / len(y_pred), 4))

import pickle
with open('/content/drive/MyDrive/MIE479/Deliverable_4/poly2_svm_2021.pkl', 'wb') as file:
    pickle.dump(poly_svm, file)

poly_vectors = poly_svm.support_vectors_
print(len(poly_vectors))

0.0142
0.2978
792544


In [ ]:
poly_dual = poly_svm.dual_coef_
print("Dual Coefficients:")
print(poly_dual)
print(len(poly_dual[0]))

poly_intercept = poly_svm.intercept_
print("Intercept:")
print(poly_intercept)

Dual Coefficients:
[[  0.50731695   0.50731695   0.50731695 ... -34.66719437 -34.66719437
  -34.66719437]]
792544
Intercept:
[-0.97106445]
